In [1]:
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [23]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.color as color
import random as r
import math
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers import Input, UpSampling2D,BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras
import argparse
import sys
#import cv2
import skimage.transform as trans
from PIL import Image
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [32]:
s=(200,200)

In [7]:
dir=os.path.join(os.getcwd(), 'Images')
image_directory = os.path.join(dir, 'resized')

In [11]:
def unet(input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2DTranspose(512,2,strides=(2,2),padding='same')(drop5)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2DTranspose(256,2,strides=(2,2),padding='same')(conv6)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2DTranspose(128,2,strides=(2,2),padding='same')(conv7)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2DTranspose(64,2,strides=(2,2),padding='same')(conv8)
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(learning_rate = 1e-4), loss='binary_crossentropy', metrics=[dice_coef])

    model.summary()
    return model

To save checkpoints

In [8]:
from keras.losses import binary_crossentropy
smooth = 1
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [9]:
filepath="cnn-parameters-improvement-{epoch:02d}-{dice_coef:.2f}"
# save the model with the best validation (development) accuracy till now
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor=[dice_coef], verbose=1, save_best_only=True, mode='max'))


In [12]:
model=unet()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [33]:
def resize_images(directory, size=(300, 300), ratio=False, metadata=False, extensions=('.jpg', '.png', '.JPG', '.PNG')):
    new_directory = os.path.join(directory, 'resized')
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    for filename in os.listdir(directory):
        if filename.endswith(extensions):
            image = Image.open(os.path.join(directory, filename))
            if not metadata:
                # Remove metadata
                data = list(image.getdata())
                image = Image.new(image.mode, image.size)
                image.putdata(data)
            if ratio:
                width, height = image.size
                new_height = int(height * size[0] / width)
                image = image.resize((size[0], new_height))
            else:
                image = image.resize(size)
            image.save(os.path.join(new_directory, filename))

In [34]:
resize_images(directory=dir, size=s, extensions=(".JPG"), ratio=False)

In [36]:
np.random.seed(12340)

In [37]:
def load_images(directory, batch_size=16):
    images = []
    labels = []
    filenames = iter(os.listdir(directory))
    for i, filename in enumerate(filenames):
        if filename.endswith(('.jpg', '.png', '.JPG', '.PNG')):
            image = tf.io.read_file(os.path.join(directory, filename))
            image = tf.image.decode_image(image, channels=3)
            image = tf.image.resize(image, s)
            image = tf.image.convert_image_dtype(image, tf.float32)
            images.append(image)
            
            label = filename.split(' ')[0]
            labels.append(label)
        
        if (i + 1) % batch_size == 0 or (i + 1) == len(os.listdir(directory)):
            images = tf.stack(images)
            labels = tf.stack(labels)
            
            yield images, labels
            images = []
            labels = []

In [38]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(labels):
    le = LabelEncoder()
    encoded_labels = le.fit_transform(labels)
    return encoded_labels

def process_images_in_batches(image_directory, batch_size=16):
    for images, labels in load_images(image_directory, batch_size=16):
        if images.shape[0] > 0: # Check if the batch is not empty
            # Encode the labels
            labels = encode_labels(labels)

            # Reshape the images to 2D
            images = tf.reshape(images, (images.shape[0], -1))

            yield images, labels
all_images = []
all_labels = []

for images, labels in process_images_in_batches(image_directory):
    if images.shape[0] > 0: # Check if the batch is not empty
        all_images.append(images)
        all_labels.append(labels)

all_images = tf.concat(all_images, axis=0)
all_labels = tf.concat(all_labels, axis=0)

# Encode the labels
all_labels = encode_labels(all_labels)

# Reshape the images to 2D
all_images = tf.reshape(all_images, (all_images.shape[0], -1))

# Balance the classes using oversampling
ros = RandomOverSampler()
balanced_images, balanced_labels = ros.fit_resample(all_images.numpy(), all_labels)

# Convert the labels to integer type
balanced_labels = balanced_labels.astype(int)

# Convert the data to TensorFlow tensors
balanced_images = tf.convert_to_tensor(balanced_images)
balanced_labels = tf.convert_to_tensor(balanced_labels)

# Reshape the images back to 4D
balanced_images = tf.reshape(balanced_images, (-1, s[0], s[1], 3))

# Split the data into training and test sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, test_indices = next(sss.split(balanced_images.numpy(), balanced_labels))

train_images = tf.gather(balanced_images, train_indices)
test_images = tf.gather(balanced_images, test_indices)
train_labels = tf.gather(balanced_labels, train_indices)
test_labels = tf.gather(balanced_labels, test_indices)

# Convert the data back to TensorFlow tensors
train_images = tf.convert_to_tensor(train_images)
test_images = tf.convert_to_tensor(test_images)
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [24]:
'''def load_images(directory, batch_size=100):
    images = []
    labels = []
    for i, filename in enumerate(os.listdir(directory)):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.JPG') or filename.endswith('.PNG'):
            image = tf.io.read_file(os.path.join(directory, filename))
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, s) # tu se isto mora prilagoditi input
            image = tf.image.convert_image_dtype(image, tf.float32)
            images.append(image)
            
            label = filename.split(' ')[0]
            labels.append(label)
        
        if (i + 1) % batch_size == 0 or (i + 1) == len(os.listdir(directory)):
            images = tf.stack(images)
            labels = tf.stack(labels)
            
            # Augmentacija udvostručuje broj slika nasumičnim rotacijama i horizontalnim preokretom
            data_augmentation = tf.keras.Sequential([
                tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
            ])
            
            augmented_images = data_augmentation(images)
            images = tf.concat([images, augmented_images], axis=0)
            labels = tf.concat([labels, labels], axis=0)
            
            yield images, labels
            images = []
            labels = []'''

In [39]:
def encode_labels(labels):
    unique_labels, _ = tf.unique(labels)
    label_to_index = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(unique_labels, tf.range(tf.size(unique_labels))),
        -1
    )
    encoded_labels = label_to_index.lookup(labels)
    return encoded_labels.numpy() # Convert to NumPy array

all_images = []
all_labels = []
for images, labels in load_images(image_directory, batch_size=16):
    if images.shape[0] > 0: # Check if the batch is not empty
        all_images.append(images)
        all_labels.append(labels)

all_images = tf.concat(all_images, axis=0)
all_labels = tf.concat(all_labels, axis=0)

# Encode the labels
all_labels = encode_labels(all_labels)

# Reshape the images to 2D
all_images = tf.reshape(all_images, (all_images.shape[0], -1))

# Balance the classes using oversampling
ros = RandomOverSampler()
balanced_images, balanced_labels = ros.fit_resample(all_images.numpy(), all_labels)

# Convert the labels to integer type
balanced_labels = balanced_labels.astype(int)

# Convert the data to TensorFlow tensors
balanced_images = tf.convert_to_tensor(balanced_images)
balanced_labels = tf.convert_to_tensor(balanced_labels)

# Reshape the images back to 4D
balanced_images = tf.reshape(balanced_images, (-1, s[0], s[1], 3))

# Split the data into training and test sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, test_indices = next(sss.split(balanced_images.numpy(), balanced_labels))

train_images = tf.gather(balanced_images, train_indices)
test_images = tf.gather(balanced_images, test_indices)
train_labels = tf.gather(balanced_labels, train_indices)
test_labels = tf.gather(balanced_labels, test_indices)

# Convert the data back to TensorFlow tensors
train_images = tf.convert_to_tensor(train_images)
test_images = tf.convert_to_tensor(test_images)
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

In [40]:
unique_train_labels, _ = tf.unique(train_labels)
num_train_labels = tf.size(unique_train_labels).numpy()
print("Number of unique labels in train set:", num_train_labels)

unique_test_labels, _ = tf.unique(test_labels)
num_test_labels = tf.size(unique_test_labels).numpy()
print("Number of unique labels in test set:", num_test_labels)

Number of unique labels in train set: 3
Number of unique labels in test set: 3


In [41]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [42]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [48]:
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model
input_shape = s[0], s[1], 3
n_classes = 3
model = densenet(input_shape,n_classes)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_251 (Conv2D)            (None, 100, 100, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_6 (MaxPooling2D)  (None, 50, 50, 64)  0           ['conv2d_251[0][0]']             
                                                                                            

In [49]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.fit(train_images, train_labels, batch_size=16, epochs=10, validation_data=(test_images, test_labels))
batch_size = 32
epochs = 100
model.fit(
    datagen.flow(train_images, train_labels, batch_size=batch_size),
    epochs=epochs,
    steps_per_epoch=len(train_images) // batch_size,
    validation_data=(test_images, test_labels)
)

Epoch 1/100
30/30 [==============================] - 205s 6s/step - loss: 2.4449 - accuracy: 0.3552 - val_loss: 101523.7500 - val_accuracy: 0.3333
Epoch 2/100
30/30 [==============================] - 185s 6s/step - loss: 1.2986 - accuracy: 0.4198 - val_loss: 4718.3354 - val_accuracy: 0.3333
Epoch 3/100
30/30 [==============================] - 187s 6s/step - loss: 1.1636 - accuracy: 0.4458 - val_loss: 1.8877 - val_accuracy: 0.3583
Epoch 4/100
30/30 [==============================] - 186s 6s/step - loss: 1.1050 - accuracy: 0.4833 - val_loss: 1.9888 - val_accuracy: 0.3333
Epoch 5/100
30/30 [==============================] - 189s 6s/step - loss: 1.0103 - accuracy: 0.5063 - val_loss: 3.6167 - val_accuracy: 0.3333
Epoch 6/100
30/30 [==============================] - 193s 6s/step - loss: 1.0199 - accuracy: 0.5063 - val_loss: 13.4042 - val_accuracy: 0.3375
Epoch 7/100
30/30 [==============================] - 192s 6s/step - loss: 0.9778 - accuracy: 0.4906 - val_loss: 39.1267 - val_accuracy: 0.2

In [43]:
# složeniji model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(s[0], s[1], 3)), # Tu promijeni size
        tf.keras.layers.MaxPooling2D((2, 2)),
        #tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    '''model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), 
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])'''
    
    return model

model2 = create_model()


#history = model2.fit(train_images, train_labels, batch_size=batch_size, epochs=100)
'''
for images, labels in load_images(os.path.join(os.getcwd(), 'resized'), batch_size=16):
    labels = encode_labels(labels)
    model.fit(images, labels, batch_size = 32, epochs=5)'''

"\nfor images, labels in load_images(os.path.join(os.getcwd(), 'resized'), batch_size=16):\n    labels = encode_labels(labels)\n    model.fit(images, labels, batch_size = 32, epochs=5)"

In [46]:
batch_size = 32
history = model2.fit(
    datagen.flow(train_images, train_labels, batch_size=batch_size),
    epochs=100,
    steps_per_epoch=len(train_images) // batch_size,
    validation_data=(test_images, test_labels)
)

Epoch 1/100
30/30 [==============================] - 23s 753ms/step - loss: 3.5695 - accuracy: 0.3229 - val_loss: 1.0986 - val_accuracy: 0.3292
Epoch 2/100
30/30 [==============================] - 22s 733ms/step - loss: 1.0986 - accuracy: 0.3302 - val_loss: 1.0984 - val_accuracy: 0.3375
Epoch 3/100
30/30 [==============================] - 23s 750ms/step - loss: 1.0989 - accuracy: 0.3271 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/100
30/30 [==============================] - 22s 713ms/step - loss: 1.0987 - accuracy: 0.3323 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/100
30/30 [==============================] - 22s 732ms/step - loss: 1.0986 - accuracy: 0.3344 - val_loss: 1.0986 - val_accuracy: 0.3375
Epoch 6/100
16/30 [===============>..............] - ETA: 9s - loss: 1.0983 - accuracy: 0.3594 

KeyboardInterrupt: 